In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from functools import partial
from sentimentanalyser.data.text import TextList, ItemList, DataBunch, SplitData
from sentimentanalyser.utils.data import Path, listify, random_splitter, compose, parallel, pad_collate, parent_labeler, read_wiki
from sentimentanalyser.data.samplers import SortishSampler, SortSampler
from sentimentanalyser.utils.preprocessing import *
from sentimentanalyser.utils.files import pickle_dump, pickle_load
from sentimentanalyser.preprocessing.processor import TokenizerProcessor, NuemericalizeProcessor, CategoryProcessor

In [3]:
path_imdb = Path("/home/anukoolpurohit/Documents/AnukoolPurohit/Datasets/imdb")

In [4]:
tl = TextList.from_files(path_imdb, folders=['train', 'test', 'unsup'])
sd_imdb = tl.split_by_func(partial(random_splitter, p=[0.9,0.1]))

In [5]:
proc_tok = TokenizerProcessor()
proc_num = NuemericalizeProcessor()
proc_cat = CategoryProcessor()

In [152]:
%time sd_imdb = sd_imdb.label_by_func(parent_labeler, proc_x=[proc_tok, proc_num], proc_y=proc_cat)


CPU times: user 29.2 s, sys: 7.44 s, total: 36.7 s
Wall time: 8min 10s


In [7]:
pickle_dump(sd_imdb, 'dumps/variable/sd_imdb.pickle')

In [8]:
sd_imdb = pickle_load('dumps/variable/sd_imdb.pickle')

In [9]:
path_wiki = Path("/home/anukoolpurohit/Documents/AnukoolPurohit/Datasets/wikitext-103")

In [10]:
train = TextList(read_wiki(path_wiki/'train.txt'), path=path_wiki)
valid = TextList(read_wiki(path_wiki/'valid.txt'), path=path_wiki)

In [11]:
len(train), len(valid)

(28476, 60)

In [12]:
sd_wiki = SplitData(train, valid)

In [13]:
proc_tok_wiki, proc_num_wiki = TokenizerProcessor(), NuemericalizeProcessor()

In [14]:
sd_wiki = sd_wiki.label_by_func(lambda x:0, proc_x=[proc_tok_wiki,proc_num_wiki], proc_y=None)

In [15]:
pickle_dump(sd_wiki, 'dumps/variable/sd_wiki.pickle')

In [15]:
sd_wiki = pickle_load('dumps/variable/sd_wiki.pickle')

In [16]:
wiki_data = sd_wiki.lm_databunchify(64, 70)

In [125]:
x1,y1 = next(iter(wiki_data.valid_dl))

In [134]:
imdb_data = sd_imdb.clas_databunchify(64)

In [135]:
x1,y1 = next(iter(imdb_data.valid_dl))

In [143]:
set([y for y in sd_imdb.train.y])

{0, 1, 2}

In [139]:
y1

tensor([1, 1, 0, 2, 2, 0, 1, 2, 2, 2, 2, 1, 2, 0, 1, 2, 0, 1, 2, 1, 2, 0, 2, 2,
        0, 2, 1, 2, 0, 0, 2, 1, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 1, 2,
        1, 2, 2, 1, 0, 1, 2, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 1, 1, 2, 2, 2,
        2, 2, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 0, 0, 2, 2, 0, 2, 1, 1, 2, 1, 2, 2,
        1, 1, 1, 2, 2, 0, 2, 2, 2, 0, 0, 1, 2, 1, 0, 2, 0, 1, 0, 1, 1, 0, 2, 1,
        2, 2, 0, 2, 2, 0, 1, 1])

In [124]:
import torch
from torch import nn
from fastai import *

In [24]:
proc_num.vocab_size

60000

In [25]:
class Model0(nn.Module):
    def __init__(self):
        self.embedding = nn.Embedding(proc_num.vocab_size, 50)
        self.fc1       = nn.Linear(50, 256)
        self.fc2       = nn.Linear(256, 2)

In [119]:
clf = SimpleRNNClassifier(proc_num.vocab_size, 10, 50, 2, 2, 0.5, 1)

In [120]:
y1 = clf(x1)

torch.Size([4, 64, 50])
torch.Size([4, 64, 50])


In [123]:
x1.shape

torch.Size([64, 70])

In [122]:
y1.shape

torch.Size([4, 64, 2])

In [90]:
proc_num.vocab

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxrep',
 'xxwrep',
 'xxup',
 'xxmaj',
 'the',
 '.',
 ',',
 'and',
 'a',
 'of',
 'to',
 'is',
 'it',
 'in',
 'I',
 'this',
 'that',
 '"',
 "'s",
 '-',
 '\n\n',
 'was',
 'as',
 'with',
 'for',
 'movie',
 'but',
 'film',
 'you',
 ')',
 'on',
 "n't",
 '(',
 'not',
 'are',
 'he',
 'his',
 'have',
 'one',
 'be',
 'all',
 'at',
 'they',
 'by',
 'an',
 'who',
 '!',
 'from',
 'so',
 'like',
 'there',
 'or',
 'just',
 'her',
 'do',
 'about',
 'has',
 'out',
 "'",
 'if',
 'what',
 'some',
 '?',
 'good',
 'when',
 'more',
 'very',
 'she',
 'up',
 'would',
 'no',
 'time',
 'even',
 'my',
 'can',
 'their',
 'which',
 'story',
 'only',
 'really',
 'see',
 'had',
 'were',
 'did',
 'well',
 'me',
 'we',
 'does',
 '...',
 'than',
 'much',
 ':',
 'could',
 'bad',
 'get',
 'been',
 'people',
 'great',
 'other',
 'also',
 'into',
 'will',
 'first',
 'because',
 'him',
 'how',
 'most',
 'them',
 'its',
 'make',
 'made',
 'then',
 'way',
 'too',
 'movies',
 'after',
